In [1]:
pip install rdkit

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pytorch_lightning

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install subword_nmt

In [5]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install deepchem

Could not fetch URL https://pypi.org/simple/deepchem/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/deepchem/ (Caused by SSLError("Can't connect to HTTPS URL because the SSL module is not available.")) - skipping
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError("Can't connect to HTTPS URL because the SSL module is not available.")) - skipping


ERROR: Could not find a version that satisfies the requirement deepchem
ERROR: No matching distribution found for deepchem


In [7]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install deepchem --user

Note: you may need to restart the kernel to use updated packages.


In [9]:
from train_cli import FsrFgLightning
from rdkit import Chem
import codecs
import numpy as np
import pandas as pd
from rdkit.Chem import Descriptors
from subword_nmt.apply_bpe import BPE

c:\Users\Prajwal\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
model = FsrFgLightning.load_from_checkpoint('epoch=22-step=5359.ckpt')
model.eval()

c:\Users\Prajwal\AppData\Local\Programs\Python\Python39\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


FsrFgLightning(
  (net): FsrFgModel(
    (encoder): Sequential(
      (0): Linear(in_features=5372, out_features=500, bias=True)
      (1): SELU()
      (2): Linear(in_features=500, out_features=100, bias=True)
    )
    (decoder): Sequential(
      (0): Linear(in_features=100, out_features=500, bias=True)
      (1): SELU()
      (2): Linear(in_features=500, out_features=5372, bias=True)
    )
    (dropout): Dropout(p=0.8, inplace=False)
    (batch_norm): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (predictor): Sequential(
      (0): Linear(in_features=100, out_features=200, bias=True)
      (1): SELU()
      (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Linear(in_features=200, out_features=100, bias=True)
      (4): SELU()
      (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Linear(in_features=100, out_features=50, bias=True)
      (7): SELU()
 

In [11]:
def smiles2index(s1, words2idx, bpe):
    t1 = bpe.process_line(s1).split()
    i1 = [words2idx[i] for i in t1]
    return i1


def index2multi_hot(i1, idx2word):
    v1 = np.zeros(len(idx2word))
    v1[i1] = 1
    return v1


def index2multi_hot_fg(molecule, fgroups_list):
    v1 = np.zeros(len(fgroups_list))
    for idx in range(len(fgroups_list)):
        if molecule.HasSubstructMatch(fgroups_list[idx]):
            v1[idx] = 1
    return v1


def smiles2vector_fgr(s1, words2idx, bpe, idx2word, fgroups_list):
    i1 = smiles2index(s1, words2idx, bpe)
    mfg = index2multi_hot(i1, idx2word)
    molecule = Chem.MolFromSmiles(s1)
    fg = index2multi_hot_fg(molecule, fgroups_list)
    return fg, mfg

In [12]:
fgroups = pd.read_csv('Functional_groups_filtered.csv')
fgroups_list = list(map(lambda x: Chem.MolFromSmarts(x), fgroups['SMARTS'].tolist()))
fgroups_list = [i for i in fgroups_list if i]
descriptor_funcs = {name: func for name, func in Descriptors.descList}

vocab_path = 'codes_drug_chembl_1500.txt'
bpe_codes_fin = codecs.open(vocab_path)
bpe = BPE(bpe_codes_fin, merges=-1, separator='')
vocab_map = pd.read_csv('subword_units_map_drug_chembl_1500.csv')
idx2word = vocab_map['index'].values
words2idx = dict(zip(idx2word, range(0, len(idx2word))))

In [28]:
f = open('saff_gen_gpt.txt')
smiles = f.read().splitlines()




In [29]:
from tqdm.auto import tqdm
import torch
sm = torch.nn.Softmax(dim = 1)
preds = []
probab = []
for smile in tqdm(smiles):
    fg, mfg = smiles2vector_fgr(smile, words2idx, bpe, idx2word, fgroups_list)
    y_pred, _ = model(fg=torch.Tensor(fg).unsqueeze(0),mfg=torch.Tensor(mfg).unsqueeze(0),num_features=None)
    probabilities = sm(y_pred) 
   # print(y_pred)
   # print(probabilities.detach().max().item())
    probab.append(probabilities.detach().max().item())
   # print(int(torch.argmax(y_pred)))
    preds.append(int(torch.argmax(y_pred)))

100%|██████████| 30/30 [00:01<00:00, 25.86it/s]


In [30]:
data = {'SMILES': smiles,'Predicted_activity':preds, 'Probability': probab}

In [31]:
pd.DataFrame(data).to_csv('moses250.csv',index=False)

In [32]:
pd.DataFrame(data)['Predicted_activity'].value_counts()

0    30
Name: Predicted_activity, dtype: int64